# Predicting a players likelihood of being drafted based on Juniors Statistics

Using Decision trees and random forest classifiers we are attempting to predict whether or not a given player will be drafted to by an NHL teams.
### Pre-analysis Prediction
I expect the results to be less accurate at predicting a drafted players likeliehood of making it to the NHL. Due to the fact the some players will play long careers in the Juniors only to never get drafted while others might only play half a season and get drafted and moved into professional leagues faster.


In [1]:
# Dependencies
from sklearn import tree
import pandas as pd
import os 
from sklearn.model_selection import train_test_split

In [39]:
# Load in data
df = pd.read_csv('data_sets/juniors_data.csv')
df.head()

,LEAGUE,PLAYER_ID,POSITION,SEASON,AGE_SEPT_15,GP,G,A,TP,PPG,...,SEASON_>20GP,CUM_GP,ONLY_1_SEASON,MIN_3_SEASONS,DY_STATUS,SHOOTS,DRAFTED,NHL_PPG,NHL_GP,NHL_DV
0,WHL,100056/tye-hand,D,2013,18.252778,31,0,2,2,0.060000,...,1,31,1,0,DY,L,No,0.0,0,0
1,OHL,100141/sean-thompson,F,2004,20.625000,38,6,5,11,0.289474,...,3,137,0,1,DY+2,L,No,0.0,0,0
2,USHL,100151/stanislav-dzakhov,F,2013,20.497222,22,3,3,6,0.272727,...,2,66,0,0,DY+2,R,No,0.0,0,0
3,QMJHL,100165/owen-bennington,F,2015,20.444444,66,2,2,4,0.060000,...,2,125,0,0,DY+2,R,No,0.0,0,0
4,QMJHL,100224/tomas-pavelka,D,2013,20.294444,56,8,31,39,0.696429,...,2,123,0,0,DY+2,L,No,0.0,0,0


In [40]:
# Dropping columns that won't impact the prediction model(player ID, season) and those which will give it the answer(NHL games played and points)
df = df.drop(['PLAYER_ID', 'SEASON', 'NHL_PPG','NHL_GP','NHL_DV','LEAGUE'], axis=1)
df.head()

,POSITION,AGE_SEPT_15,GP,G,A,TP,PPG,SEASON_NO,SEASON_>20GP,CUM_GP,ONLY_1_SEASON,MIN_3_SEASONS,DY_STATUS,SHOOTS,DRAFTED
0,D,18.252778,31,0,2,2,0.060000,1,1,31,1,0,DY,L,No
1,F,20.625000,38,6,5,11,0.289474,3,3,137,0,1,DY+2,L,No
2,F,20.497222,22,3,3,6,0.272727,2,2,66,0,0,DY+2,R,No
3,F,20.444444,66,2,2,4,0.060000,2,2,125,0,0,DY+2,R,No
4,D,20.294444,56,8,31,39,0.696429,2,2,123,0,0,DY+2,L,No


In [41]:
# Convert Position to int (Forward = 0, Defense = 1, Goalie= 2)
df.POSITION[df.POSITION == 'F'] = 0
df.POSITION[df.POSITION == 'D'] = 1
df.POSITION[df.POSITION == 'G'] = 2

# Convert shooting side to int (left = 0, right = 1)
df.SHOOTS[df.SHOOTS == 'L'] = 0
df.SHOOTS[df.SHOOTS == 'R'] = 1
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\

,POSITION,AGE_SEPT_15,GP,G,A,TP,PPG,SEASON_NO,SEASON_>20GP,CUM_GP,ONLY_1_SEASON,MIN_3_SEASONS,DY_STATUS,SHOOTS,DRAFTED
0,1,18.252778,31,0,2,2,0.060000,1,1,31,1,0,DY,0,No
1,0,20.625000,38,6,5,11,0.289474,3,3,137,0,1,DY+2,0,No
2,0,20.497222,22,3,3,6,0.272727,2,2,66,0,0,DY+2,1,No
3,0,20.444444,66,2,2,4,0.060000,2,2,125,0,0,DY+2,1,No
4,1,20.294444,56,8,31,39,0.696429,2,2,123,0,0,DY+2,0,No


In [42]:
# Convert Draft year to int (draft year = 2, dy-2 = 0, etc)
df.DY_STATUS[df.DY_STATUS == 'DY-3'] = 0
df.DY_STATUS[df.DY_STATUS == 'DY-2'] = 1
df.DY_STATUS[df.DY_STATUS == 'DY-1'] = 2
df.DY_STATUS[df.DY_STATUS == 'DY'] = 3
df.DY_STATUS[df.DY_STATUS == 'DY+1'] = 4
df.DY_STATUS[df.DY_STATUS == 'DY+2'] = 5
df.DY_STATUS[df.DY_STATUS == 'DY+3'] = 6
df.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\

,POSITION,AGE_SEPT_15,GP,G,A,TP,PPG,SEASON_NO,SEASON_>20GP,CUM_GP,ONLY_1_SEASON,MIN_3_SEASONS,DY_STATUS,SHOOTS,DRAFTED
0,1,18.252778,31,0,2,2,0.060000,1,1,31,1,0,3,0,No
1,0,20.625000,38,6,5,11,0.289474,3,3,137,0,1,5,0,No
2,0,20.497222,22,3,3,6,0.272727,2,2,66,0,0,5,1,No
3,0,20.444444,66,2,2,4,0.060000,2,2,125,0,0,5,1,No
4,1,20.294444,56,8,31,39,0.696429,2,2,123,0,0,5,0,No
5,0,17.822222,21,0,1,1,0.050000,1,1,21,1,0,2,0,No
6,0,21.605556,71,21,28,49,0.690141,4,4,284,0,1,6,0,No
7,0,20.572222,8,1,0,1,0.130000,1,0,8,1,0,5,1,No
8,1,20.777778,59,8,12,20,0.338983,2,1,78,0,0,5,1,No
9,1,20.566667,51,3,9,12,0.235294,3,3,182,0,1,5,1,No


In [53]:
# Getting an error where a '-' can't be converted to int to fit the model. So I'm converting to NaN and dropping those rows
df.convert_objects(convert_numeric=True)
df.convert_objects(convert_numeric=True).dropna()

df.eq('-')
df = df[~df.eq('-').any(1)]
df

df.eq('D')
df = df[~df.eq('D').any(1)]
df

df.dropna()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,POSITION,AGE_SEPT_15,GP,G,A,TP,PPG,SEASON_NO,SEASON_>20GP,CUM_GP,ONLY_1_SEASON,MIN_3_SEASONS,DY_STATUS,SHOOTS,DRAFTED
0,1,18.252778,31,0,2,2,0.060000,1,1,31,1,0,3,0,No
1,0,20.625000,38,6,5,11,0.289474,3,3,137,0,1,5,0,No
2,0,20.497222,22,3,3,6,0.272727,2,2,66,0,0,5,1,No
3,0,20.444444,66,2,2,4,0.060000,2,2,125,0,0,5,1,No
4,1,20.294444,56,8,31,39,0.696429,2,2,123,0,0,5,0,No
5,0,17.822222,21,0,1,1,0.050000,1,1,21,1,0,2,0,No
6,0,21.605556,71,21,28,49,0.690141,4,4,284,0,1,6,0,No
7,0,20.572222,8,1,0,1,0.130000,1,0,8,1,0,5,1,No
8,1,20.777778,59,8,12,20,0.338983,2,1,78,0,0,5,1,No
9,1,20.566667,51,3,9,12,0.235294,3,3,182,0,1,5,1,No


In [54]:
# split data into inputs and outputs
data = df.values
inputs = data[:, 0:14]
outputs = data[:, 14]

In [55]:
# split data into inputs and outputs
outcomes = df['DRAFTED']
outcomes.head()

input_factors = df.drop(['DRAFTED'], axis = 1)
input_factors.head()
feature_names = input_factors.columns

In [56]:
# Create Train/test groups
input_train, input_test, output_train, output_test = train_test_split(input_factors, outcomes, random_state=42)

In [57]:
# Fit and test decision tree model
clf = tree.DecisionTreeClassifier()
clf = clf.fit(input_train, output_train)
clf.score(input_test, output_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').